In [1]:
import pandas as pd
import numpy as np
import pymongo
import math

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
def is_not_nan(value):
    if type(value) != str:
        if math.isnan(value):
            return False
        else:
            return True
    else:
        return True

In [6]:
db = client.student_db

In [7]:
student_collection = db.students

In [9]:
students = pd.read_csv("McSQL Data - students.csv")
classes = pd.read_csv("McSQL Data - classes.csv")
countries = pd.read_csv("McSQL Data - countries.csv")
xtras = pd.read_csv("McSQL Data - extracurriculars.csv")

In [11]:
classes = classes.set_index('class_id')
xtras = xtras.set_index('xtra_id')
countries = countries.set_index('country_id')

In [12]:
classes.credits = classes.credits.astype('object')
xtras.application = xtras.application.astype('object')
countries.population = countries.population.astype('object')
countries.gdp_per_capita = countries.gdp_per_capita.astype('object')

In [13]:
master_list = []

for index in range(0, len(students)):
    student = students.iloc[index]    
    student_dictionary = {}
    for i in range(3):
        student_dictionary[student.index[i]] = student[i]
    fav_class = student[3]
    fav_class_dict = {}
    fav_class_list = classes.loc[fav_class]
    for i in range(0, 3):
        fav_class_dict[fav_class_list.index[i]] = fav_class_list[i]
        
    prereq_list = []
    if is_not_nan(fav_class_list[3]):
        prereq_list.append(fav_class_list[3])
    if is_not_nan(fav_class_list[4]):
        prereq_list.append(fav_class_list[4])      
    fav_class_dict['prereqs'] = prereq_list
        
    for i in range(5, len(fav_class_list)):
        fav_class_dict[fav_class_list.index[i]] = fav_class_list[i]
    fav_class_dict
    student_dictionary[student.index[3]] = fav_class_dict

    final_list = []
    xtra_list = []
    for i in range(4, 7):
        if is_not_nan(student[i]):
            xtra_list.append(student[i])
    xtra_list
    for j in xtra_list:
        xtra_dict = {}
        xtra_list = xtras.loc[j]
        for i in range(0, len(xtra_list)):
            xtra_dict[xtra_list.index[i]] = xtra_list[i]
        final_list.append(xtra_dict)
    student_dictionary['xtra_id'] = final_list

    fav_vac = student[7]
    fav_vac_dict = {}
    fav_vac_list = countries.loc[fav_vac]
    for i in range(0, len(fav_vac_list)):
        fav_vac_dict[fav_vac_list.index[i]] = fav_vac_list[i]
    student_dictionary[student.index[7]] = fav_vac_dict

    dream_vac = student[8]
    dream_vac_dict = {}
    dream_vac_list = countries.loc[dream_vac]
    for i in range(0, len(dream_vac_list)):
        dream_vac_dict[dream_vac_list.index[i]] = dream_vac_list[i]
    student_dictionary[student.index[8]] = dream_vac_dict
    
    master_list.append(student_dictionary)

In [15]:
student_collection.insert_many(master_list)